In [7]:
import os
import sys
import warnings

import pandas as pd
import numpy as np

from sklearn.model_selection import  train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

import mlflow
import mlflow.sklearn


In [9]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [10]:
remote_server_url="http://127.0.0.1:5000"
mlflow.set_tracking_uri(remote_server_url)

In [11]:
exp_name="Elastic_net_wine"
mlflow.set_experiment(exp_name)

2023/01/29 10:52:19 INFO mlflow.tracking.fluent: Experiment with name 'Elastic_net_wine' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/496180605469689835', creation_time=1674969739076, experiment_id='496180605469689835', last_update_time=1674969739076, lifecycle_stage='active', name='Elastic_net_wine', tags={}>

In [34]:
def load_data(data_path):
    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    
    data = pd.read_csv(data_path)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    return train_x,test_x,train_y,test_y
def train(alpha=0.5,l1_ratio=0.5):
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    data_path=r"C:\Users\sudheer.tetali\Downloads\wine-quality.csv"
    train_x,test_x,train_y,test_y=load_data(data_path)
    with mlflow.start_run():
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        predicted_qualities = lr.predict(test_x)

        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        mlflow.log_artifact(data_path)
        print("save to {}".format(mlflow.get_artifact_uri()))

        mlflow.sklearn.log_model(lr, "model")

In [35]:
train(0.5,0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284975954
  MAE: 0.6278761410160693
  R2: 0.12678721972772677
save to mlflow-artifacts:/496180605469689835/a7ac59f545184f6a9f50e1963dcb807d/artifacts


In [36]:
train(0.2,0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7859129997062341
  MAE: 0.6155290394093895
  R2: 0.20224631822892103
save to mlflow-artifacts:/496180605469689835/789cf865d481487ca58ad3d4550de63c/artifacts


In [37]:
train(0.1,0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7792546522251949
  MAE: 0.6112547988118587
  R2: 0.2157063843066196
save to mlflow-artifacts:/496180605469689835/23d77a81208e484e9ea36f68bacf9d32/artifacts
